 If we have caliper, then we can limit the spinner flow rate analysis to only those intervals that are in gauge

 Jorge Acuna paper at stanford workshop

 auto removal of data that are inconsistant with what would be expected (for instance the spinner was stuck)

## Understanding spinner data

The spinner tool's impeller rotates in the positive direction when the tool is pulled upwards through the well and in the negitive direction when the tool is lowered down into the well. 

If the tool is moving at the same rate as the injected fluid, then the impeller stops spinner (frequency = 0)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interactive, Layout, FloatSlider
from utilities import* # functions in the utilities.py file

Because the cell below includes all steps required to import and mudge the data (everything from notebook 1), it will take a little while to run. 

In [ ]:
flowrate = read_flowrate(r'Data-FlowRate.xlsx')
pts = read_pts(r'Data-PTS.xlsx')

In [ ]:
pts.shape

## Add flowrate values to the pts dataframe

Our fluid velocity analysis requires that we know the pump rate and spinner frequency. There are several ways we could approch this:

1. We could assume that the pump rate was held perfectly steady the defined pump rate and set a single value
2. We could use the actual flowrate data if that is available

As have good quality surface pupmp data, we use the bespoke function below to append the flowrate data to the pts dataframe. As the flowrate data is recorded at a courser time resolution than the pts data. We used linear interpolation to fill the gaps. 

If you are using this workflow on your own data, you need to adjust the column names in the function. This method is documented in bonus-combine-date.ipynb

In [ ]:
pts = append_flowrate_to_pts(flowrate, pts)

In [ ]:
pts.shape

In [ ]:
pts.head(2)

## Look at the spinner data -- by depth

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(24,8),sharey=True)

ax1.scatter(pts.frequency_hz, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)
ax2.scatter(pts.datetime, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax1.set_ylim(1000,0)
ax1.set_xlim(-30,30)

ax1.set_ylabel('Depth [m]')
ax1.set_xlabel('Spinner frequency [hz]')

ax2.set_xlabel('Time [hh:mm]')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax3.set_ylabel('Flowrate [t/hr]')

for ax in [ax1, ax2]:
    ax.grid()

## Look at the spinner data -- by time

As we zoom in, we see that the issue of the slow-down

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(28,8))

#ax1.plot(pts.datetime, pts.frequency_hz, c='k', linestyle = '-', linewidth = 1, alpha = 0.3)
ax1.scatter(pts.datetime, pts.frequency_hz, c = pts.timestamp, s = 5, linewidths = 0)
ax2.scatter(pts.datetime, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax4 = ax1.twinx()
ax4.plot(pts.datetime, pts.depth_m, 
    c='k', linestyle = '-', linewidth = 2, alpha = 0.3, 
    label='Tool depth [m]')
ax4.set_ylim(1000,-1000)
ax4.set_ylabel('Tool depth [m]')

ax1.set_ylim(-30,30)
ax1.set_ylabel('Spinner frequency [hz]')

ax2.set_ylim(1000,0)
ax2.set_ylabel('Tool depth [m/s]')

for ax in [ax1,ax2]:
    ax.set_xlabel('Time [hh:mm]')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax3.set_ylabel('Flowrate [t/hr]')

for ax in [ax1, ax2]:
    ax.grid()

#
# Limit to a time range
#

start_time = pd.to_datetime('2020-12-11 09:30:00')
end_time = pd.to_datetime('2020-12-11 10:30:00')

ax1.set_xlim(start_time,end_time)
ax4.set_ylim(1000,400);

### Remove data aquired when the tool is stationary or slowing and the data aquired in the casing

To understand this method and how we decided which data to filter, refer to bonus-filter-by-toolspeed.ipynb

In [ ]:
moving_pts = pts[
    (pts.speed_mps > 0.9 ) & (pts.speed_mps < pts.speed_mps.max()) | 
    (pts.speed_mps > pts.speed_mps.min() ) & (pts.speed_mps < -0.9)
    ]

production_shoe = 462.5  

clean_pts = moving_pts[(moving_pts.depth_m < moving_pts.depth_m.max()) & (moving_pts.depth_m > production_shoe)]

clean_pts.describe()

In [ ]:
## Plot clean data

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(28,8))

ax1.scatter(clean_pts.datetime, clean_pts.frequency_hz, c = clean_pts.timestamp, s = 5, linewidths = 0)
ax2.scatter(clean_pts.datetime, clean_pts.depth_m, c = clean_pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax4 = ax1.twinx()
ax4.plot(pts.datetime, pts.depth_m, 
    c='k', linestyle = '-', linewidth = 2, alpha = 0.3, 
    label='Tool depth [m]')
ax4.set_ylim(1000,400)
ax4.set_ylabel('Tool depth [m]')

ax1.set_ylim(-30,30)
ax1.set_ylabel('Spinner frequency [hz]')

ax2.set_ylim(1000,0)
ax2.set_ylabel('Tool depth [m]')

for ax in [ax1,ax2]:
    ax.set_xlabel('Time [hh:mm]')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

ax3.set_ylabel('Flowrate [t/hr]')

for ax in [ax1, ax2]:
    ax.grid()

#
# Limit to a time range
#

start_time = pd.to_datetime('2020-12-11 14:30:00')
end_time = pd.to_datetime('2020-12-11 15:30:00')

ax1.set_xlim(start_time,end_time);

## Select data by rate



In [ ]:
from ipywidgets import interactive, Layout, FloatSlider

min_timestamp = pts.timestamp.iloc[0]
max_timestamp = pts.timestamp.iloc[-1]

def subselect_plot(start_value, stop_value):
    f,ax = plt.subplots(1,1, figsize = (20,6))
    ax.scatter(clean_pts.timestamp, clean_pts.depth_m,
        c = 'k', s = 1, linewidths = 0, label = 'Tool depth')
    ax1 = ax.twinx()
    ax1.plot(flowrate.timestamp, flowrate.flow_tph, 
        ':', c='k', label='Surface pump flowrate')
    ymin = pts.depth_m.min()
    ymax = pts.depth_m.max() + 100
    ax.vlines(start_value, ymin, ymax, color='tab:green')
    ax.vlines(stop_value, ymin, ymax, color='tab:red')
    ax.set_ylim(pts.depth_m.max() + 100, 0)
    ax.set_xlabel('Timestamp [sec]')
    ax.set_ylabel('Tool depth [m]')
    ax1.set_ylabel('Flowrate [t/hr]')

result = interactive(subselect_plot,
         
         start_value = FloatSlider
         (
             value = (max_timestamp - min_timestamp)/3 + min_timestamp,
             description = 'start',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=False,
             layout = Layout(width='80%'),
             ),
          
          stop_value = FloatSlider
          (
             value = (max_timestamp - min_timestamp)/2 + min_timestamp, 
             description = 'stop',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=False,
             layout = Layout(width='80%')
             )
)

display(result);


In [ ]:
print(
    'start =',result.children[0].value, ' which is', datetime.fromtimestamp(result.children[0].value), 
    '\n stop =', result.children[1].value, ' which is', datetime.fromtimestamp(result.children[1].value),
    )

### Record your analysis

As part of making this process repeatable and so that it is easy to come back later to check the analysis, you need to use the range you have selected to manually define objects in the next step.

Copy-paste the timestamps printed above into the cell below. They are now the objects that define the start and stop points for splicing the moving_pts dataframe. 

We could have defined the start and stop objects as the result.childern\[0\].value and result.childern\[1\].value objects. But if we did this, you would lose your work because these ipywidget results change every time the sliders are moved or the notebook is re-run. 

Second rate


start = 1607641584.448  which is 2020-12-11 12:06:24.448000 

stop = 1607644334.448  which is 2020-12-11 12:52:14.448000

In [ ]:
start = 1607641584.448  # paste your start value here
stop = 1607644334.448   # paste your stop value here
name = 'Second pump rate'  # create an informative title for the data

pts_second_rate = clean_pts[
    (clean_pts.datetime > datetime.fromtimestamp(start)) 
    & (clean_pts.datetime < datetime.fromtimestamp(stop))
]

flowrate_second_rate = flowrate[
    (flowrate.datetime > datetime.fromtimestamp(start)) 
    & (flowrate.datetime < datetime.fromtimestamp(stop))
]

In [ ]:
# import the first heating temprature log for comparison

heating_0days = pd.read_csv('data-heating0days.csv')
heating_0days.head(2)

## Consider grabbing the second temp profile and throwing it in this plot

Or all of them??

In [ ]:
fig, (ax1, ax3) = plt.subplots(1, 2,figsize=(24,8))

ax1.scatter(pts_second_rate.datetime, pts_second_rate.frequency_hz, 
    c = pts_second_rate.timestamp, s = 5, linewidths = 0)

ax2 = ax1.twinx()
ax2.plot(flowrate_second_rate.datetime, flowrate_second_rate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax3.scatter(pts_second_rate.frequency_hz, pts_second_rate.depth_m, 
    c = pts_second_rate.timestamp, s = 5, linewidths = 0)

ax4 = ax3.twiny()
ax4.plot(heating_0days.temp_degC, heating_0days.depth_m,
    c = 'k', linestyle = '-', linewidth = 3, alpha = 0.3,
    label = 'First heating run')

ax1.set_ylabel('Spinner frequency [hz]')
ax1.set_xlabel('Time [hh:mm]')
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax2.set_ylabel('Pump rate [t/hr]')

ax3.set_xlabel('Spinner frequency [hz]')
ax3.set_ylabel('Depth [m]')

ax4.set_xlabel('Temperature [degC]')
ax4.set_xlim(0,200)

for ax in [ax1,ax3]:
    ax.grid()

for ax in [ax2, ax4]:
    ax.legend()

In [ ]:
pts_second_rate.columns

In [ ]:
fig, (ax) = plt.subplots(1, 1,figsize=(8,8))

ax.scatter(pts_second_rate.interpolated_flow_tph, pts_second_rate.frequency_hz,
    c = pts_second_rate.timestamp, s = 30, linewidths = 0)

ax.set_ylabel('Spinner frequency [hz]')
ax.set_xlabel('Flow rate [t/hr]')


In [ ]:
this_depth_df = pts_second_rate[
    (pts_second_rate.depth_m > 700 ) & (pts_second_rate.depth_m < 701)
    ]

In [ ]:
fig, (ax) = plt.subplots(1, 1,figsize=(8,8))

ax.scatter(this_depth_df.frequency_hz, this_depth_df.speed_mps,
    color = 'none', edgecolors = 'k', marker='o', s = 50, linewidths = 1)

ax.hlines(0, -40, 40, color = 'k', linewidth = 0.5)
ax.vlines(0, -2, 2, color = 'k', linewidth = 0.5)

ax.set_xlim(-40,40)
ax.set_ylim(-2,2)

ax.set_xlabel('Spinner frequency [hz]')
ax.set_ylabel('Tool speed [mps]')

ax.grid()

# Goal

For at least two fluid injection rates, generate cross-plots to calculate the fluid velocity profile

You have finished the T21 geothermal well completion test tutoral. Well done!

***

<p><center>© 2021 <a href="https://www.cubicearth.nz/">Irene Wallis</a> and <a href="https://www.linkedin.com/in/katie-mclean-25994315/">Katie McLean</a> <a href="https://creativecommons.org/licenses/by/4.0/"</a></center></p>

<p><center>Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a></center></p>

<p><center>Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.</center></p>

***